In [16]:
import open3d as o3d
import numpy as np
def get_tree_from_coord(pcd, grd_pcd, coord:list, expand_x_y:list=[10.0,10.0], expand_z:list=[-10.0,10.0]):
    # CAREFUL THE Y IS ACTUALLY NEGATIVE
    xc, yc = coord[0], -coord[1]
    
    l,w = expand_x_y
    zmin, zmax = expand_z
    bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(xc-l/2,yc-w/2,zmin),max_bound=(xc+l/2,yc+w/2,zmax))
    ground = grd_pcd.crop(bbox)
    
    # Removing Outlier by taking the minimum z value of Ground from CSF FILTER
    zmin = ground.get_min_bound()[2]
    zmin_tolerance = ground.get_max_bound()[2] - 2.0
    zmin = zmin_tolerance if zmin > zmin_tolerance else zmin
    bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(xc-l/2,yc-w/2,zmin),max_bound=(xc+l/2,yc+w/2,zmax))
    tree = pcd.crop(bbox) + ground
    return tree

In [17]:
pcd = o3d.io.read_point_cloud("model_dense4k10fps1Tree.ply", format='auto')

In [ ]:
center = pcd.get_center()[0:2]
center[1]*=-1
side_view_img_size = [320, 768]
side_view_step_size = 0.02
ex_w, ex_h = (dim*side_view_step_size for dim in side_view_img_size)

almost_tree = get_tree_from_coord(pcd, pcd, center, expand_x_y=[ex_w,ex_w], 
                                                    expand_z=[-100, 100]
                                                    )

In [43]:
non_grd = pcd
# Expand search radius of new center by 2 meters with how many linspace?
expand_meters = 2
x_min_pcd, y_min_pcd, z_min = non_grd.get_min_bound()
x_max_pcd, y_max_pcd, z_max = non_grd.get_max_bound()

new_min, new_max = center-expand_meters/2 , center+expand_meters/2

# Find new x_coords
new_x_list, x_increment= np.linspace(new_min[0], new_max[0], 4, retstep=True)
new_y_list, y_increment = np.linspace(new_min[1], new_max[1], 4, retstep=True)

for i, new_x in enumerate(new_x_list):
    for i, new_y in enumerate(new_y_list):
        new_center = (new_x, new_y)
        almost_tree = get_tree_from_coord(
            pcd, 
            pcd, 
            new_center, 
            expand_x_y=[ex_w,ex_w], 
            expand_z=[-100, 100]
            )
        o3d.visualization.draw_geometries([almost_tree])

KeyboardInterrupt: 

In [ ]:
o3d.visualization.draw_geometries([almost_tree])

In [42]:
print(ex_w,ex_h)

6.4 15.36


In [ ]:
import open3d as o3d
import numpy as np
import os
from PIL import Image
import imageio

def custom_draw_geometry_with_rotation(filename, filetype):
    input_file = f"{filename}"+filetype
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=800, height=600, visible=True)
    vis.add_geometry(pcd)
    opt = vis.get_render_option()
    opt.point_size = 0.1
    opt.background_color = np.asarray([0.0, 0.0, 0.0])
    ctr = vis.get_view_control()
    ctr.set_front([0, 1, 0])  # Looking along Y
    ctr.set_up([0, 0, 1])
    ctr.set_lookat(pcd.get_center())
    
    images = []
    for i in range(360):
        # Rotate 360 degrees in total

        ctr.rotate(4, 0)  # Rotate around Z-axis
        
        # Update renderer
        vis.poll_events()
        vis.update_renderer()
        img = vis.capture_screen_float_buffer(do_render=False)
        img = (np.asarray(img) * 255).astype(np.uint8)
        images.append(img)
    vis.destroy_window()
    
    imageio.mimsave(f"{filename}.mp4", images, fps=60)
if __name__ == "__main__":
    # Example usage
    input_file = "point_cloud_ori.ply"  # Change to your point cloud file
    pcd = o3d.io.read_point_cloud(input_file)
    custom_draw_geometry_with_rotation(pcd)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
